In [1]:
import os
os.environ["DISPLAY"] = ":0.0"

import awkward as ak
import uproot
import ROOT
ROOT.gROOT.SetBatch(False)
import numpy as np
import time
from datetime import datetime
%jsroot on

def close_canvas(canvas_name="c"):
    """Fecha um canvas de forma segura"""
    try:
        c = ROOT.gROOT.FindObject(canvas_name)
        if c and c.InheritsFrom("TCanvas"):
            c.Close()
            return True
        return False
    except:
        return False

# Uso:
close_canvas("c")  # Fecha um canvas específico

False

In [2]:
import uproot
import ROOT
import numpy as np
import time
from datetime import datetime

# ---------------- Funções ----------------
def compute_invariant_mass(pt1, eta1, phi1, mass1, pt2, eta2, phi2, mass2):
    """Calcula massa invariante de duas partículas"""
    px1 = pt1 * np.cos(phi1)
    py1 = pt1 * np.sin(phi1)
    pz1 = pt1 * np.sinh(eta1)
    E1 = np.sqrt(px1**2 + py1**2 + pz1**2 + mass1**2)

    px2 = pt2 * np.cos(phi2)
    py2 = pt2 * np.sin(phi2)
    pz2 = pt2 * np.sinh(eta2)
    E2 = np.sqrt(px2**2 + py2**2 + pz2**2 + mass2**2)

    E = E1 + E2
    px = px1 + px2
    py = py1 + py2
    pz = pz1 + pz2
    
    return np.sqrt(max(E**2 - (px**2 + py**2 + pz**2), 0.0))

def compute_4body_invariant_mass(pt1, eta1, phi1, mass1, pt2, eta2, phi2, mass2, 
                                pt3, eta3, phi3, mass3, pt4, eta4, phi4, mass4):
    """Calcula massa invariante de quatro partículas"""
    def four_vector(pt, eta, phi, mass):
        px = pt * np.cos(phi)
        py = pt * np.sin(phi)
        pz = pt * np.sinh(eta)
        E  = np.sqrt(px**2 + py**2 + pz**2 + mass**2)
        return E, px, py, pz

    E1, px1, py1, pz1 = four_vector(pt1, eta1, phi1, mass1)
    E2, px2, py2, pz2 = four_vector(pt2, eta2, phi2, mass2)
    E3, px3, py3, pz3 = four_vector(pt3, eta3, phi3, mass3)
    E4, px4, py4, pz4 = four_vector(pt4, eta4, phi4, mass4)

    E  = E1 + E2 + E3 + E4
    px = px1 + px2 + px3 + px4
    py = py1 + py2 + py3 + py4
    pz = pz1 + pz2 + pz3 + pz4

    return np.sqrt(max(E**2 - (px**2 + py**2 + pz**2), 0.0))

def deltaR(eta1, phi1, eta2, phi2):
    """Calcula ΔR entre duas partículas"""
    deta = eta1 - eta2
    dphi = phi1 - phi2
    while dphi > np.pi:
        dphi -= 2 * np.pi
    while dphi < -np.pi:
        dphi += 2 * np.pi
    return np.sqrt(deta**2 + dphi**2)

# ---------------- Configuração ----------------
file_paths = [
    "DatasetNanoAOD/C0014C26-5B84-1F41-BEC4-4283C33952A9.root",
    "DatasetNanoAOD/2B7EEF60-337B-2341-AF48-B681E2BFF486.root",
    "DatasetNanoAOD/3E946E36-0338-C042-9005-CC3111C92352.root",
    "DatasetNanoAOD/4FF75944-DB7D-904B-A62A-66C0DABDA865.root",
    "DatasetNanoAOD/5B6B013D-C511-FF4A-AE75-70BE4F3EFCED.root",
    "DatasetNanoAOD/6AB71536-CA49-DD4B-B31A-29B7CFA16F4A.root",
    "DatasetNanoAOD/8A946F5C-0C68-614C-902F-E9227FCC2471.root",
    "DatasetNanoAOD/8CB22E10-7CC7-864A-A990-1C6C9F672FB5.root",
    "DatasetNanoAOD/8EB8AE5C-24CE-1A4B-9623-99F4DA934540.root",
    "DatasetNanoAOD/9E921749-9FDA-B74D-AE2F-130566FB049E.root",
    "DatasetNanoAOD/72EEC211-1B4A-404A-9A8D-C6627A9DAF9B.root",
    "DatasetNanoAOD/75A3A969-CA67-D94C-B929-D488ECECCAC5.root",
    "DatasetNanoAOD/78A279DD-780C-3748-A14A-9D363AD8AF9A.root",
    "DatasetNanoAOD/647E1671-C248-4343-B12D-7300BFA2F336.root",
    "DatasetNanoAOD/57939F18-B39F-9243-8E06-BE0EBE1CF6D3.root",
    "DatasetNanoAOD/A3CD7333-5E7B-054C-B88E-A1922F40F74E.root",
    "DatasetNanoAOD/B169CF10-49A5-4248-AC0C-7ABD1CAE477F.root",
    "DatasetNanoAOD/B1868DE2-4814-3A45-8BF1-BA339CFE0611.root",
    "DatasetNanoAOD/BE696517-BE1F-854E-B8A9-AFE636B71EE7.root",
    "DatasetNanoAOD/C1FC38AF-BE29-F444-910B-5036C0FEBBB2.root",
    "DatasetNanoAOD/C06CAC81-F0AD-9746-A4D2-CCD09788D9DC.root",
    "DatasetNanoAOD/C50E1375-45BB-EB4F-8594-28BECE218F38.root",
    "DatasetNanoAOD/C357DA0F-F15F-3244-BB4D-7B2185F441E2.root",
    "DatasetNanoAOD/D10D3D01-3440-134D-8705-4BE4C790A7F6.root",
    "DatasetNanoAOD/F165AB77-19B5-4040-BED1-6DEFDCB246B5.root",
]
output_root = "bs_reconstruction.root"

# ---------------- ROOT output ----------------
output_file = ROOT.TFile(output_root, "RECREATE")

# Histogramas para φ → K⁺K⁻
hist_phi_mass = ROOT.TH1F("hist_phi_mass", "Massa invariante #phi → K⁺K⁻;Massa [GeV];Eventos", 100, 0.9, 1.2)

# Histogramas para J/ψ → μ⁺μ⁻
hist_jpsi_mass = ROOT.TH1F("hist_jpsi_mass", "Massa invariante J/#psi → #mu⁺#mu⁻;Massa [GeV];Eventos", 100, 2.8, 3.4)

# Histogramas para Bₛ → J/ψφ
hist_bs_mass = ROOT.TH1F("hist_bs_mass", "Massa invariante B_{s} → J/#psi #phi;Massa [GeV];Eventos", 100, 4.5, 6.0)

# Histogramas de controle
hist_k_pt = ROOT.TH1F("hist_k_pt", "pT dos kaons;pT [GeV];Kaons", 100, 0, 10)
hist_mu_pt = ROOT.TH1F("hist_mu_pt", "pT dos múons;pT [GeV];Múons", 100, 0, 10)
hist_deltaR_phi = ROOT.TH1F("hist_deltaR_phi", "#Delta R entre kaons;#Delta R;Pares", 100, 0, 1)
hist_deltaR_jpsi = ROOT.TH1F("hist_deltaR_jpsi", "#Delta R entre múons;#Delta R;Pares", 100, 0, 1)

# NOVOS histogramas: massa reconstruída de candidatos individuais
hist_mu_mass = ROOT.TH1F("hist_mu_mass", "Massa dos candidatos a múon;Massa [GeV];Eventos", 100, 0, 0.2)
hist_k_mass = ROOT.TH1F("hist_k_mass", "Massa dos candidatos a kaon;Massa [GeV];Eventos", 100, 0.4, 0.6)

# Constantes
MASS_K = 0.493677    # Massa do kaon em GeV
MASS_MU = 0.105658   # Massa do múon em GeV
MASS_JPSI = 3.0969   # Massa do J/ψ em GeV
MASS_PHI = 1.019461  # Massa do φ em GeV

# ---------------- Processamento ----------------
print(f"Iniciando processamento em {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
start_time = time.time()

total_events = 0
events_with_phi = 0
events_with_jpsi = 0
events_with_bs = 0

for i, file_path in enumerate(file_paths, 1):
    print(f"\nArquivo {i}/{len(file_paths)}: {file_path.split('/')[-1]}")
    print(f"    Eventos com φ: {events_with_phi}, com J/ψ: {events_with_jpsi}, com Bₛ: {events_with_bs}")
    try:
        with uproot.open(file_path) as file:
            events = file["Events"]
            
            # Variáveis para kaons (IsoTrack)
            track_pt_arr = events["IsoTrack_pt"].array()
            track_eta_arr = events["IsoTrack_eta"].array()
            track_phi_arr = events["IsoTrack_phi"].array()
            track_charge_arr = events["IsoTrack_charge"].array()
            track_dxy_arr = events["IsoTrack_dxy"].array()
            track_dz_arr = events["IsoTrack_dz"].array()

            
            # Variáveis para múons
            muons_pt = events["Muon_pt"].array()
            muons_eta = events["Muon_eta"].array()
            muons_phi = events["Muon_phi"].array()
            muons_mass = events["Muon_mass"].array()
            muons_charge = events["Muon_charge"].array()

            # Loop nos eventos
            for ievt in range(len(track_pt_arr)):
                total_events += 1

                # --- Preenche histograma de massa dos candidatos a kaon ---
                for itrk in range(len(track_pt_arr[ievt])):
                    pt = float(track_pt_arr[ievt][itrk])
                    eta = float(track_eta_arr[ievt][itrk])
                    phi = float(track_phi_arr[ievt][itrk])
                    charge = int(track_charge_arr[ievt][itrk])
                    if charge == 0:
                        continue
                    px = pt * np.cos(phi)
                    py = pt * np.sin(phi)
                    pz = pt * np.sinh(eta)
                    E  = np.sqrt(px**2 + py**2 + pz**2 + MASS_K**2)
                    inv_mass = np.sqrt(max(E**2 - (px**2 + py**2 + pz**2), 0.0))
                    hist_k_mass.Fill(inv_mass)

                # --- Preenche histograma de massa dos candidatos a múon ---
                for imu in range(len(muons_pt[ievt])):
                    pt = float(muons_pt[ievt][imu])
                    eta = float(muons_eta[ievt][imu])
                    phi = float(muons_phi[ievt][imu])
                    px = pt * np.cos(phi)
                    py = pt * np.sin(phi)
                    pz = pt * np.sinh(eta)
                    E  = np.sqrt(px**2 + py**2 + pz**2 + MASS_MU**2)
                    inv_mass = np.sqrt(max(E**2 - (px**2 + py**2 + pz**2), 0.0))
                    hist_mu_mass.Fill(inv_mass)
                
                # --- RECONSTRUÇÃO DO φ → K⁺K⁻ ---
                kaons_plus = []
                kaons_minus = []
                phi_candidates = []
                
                for itrk in range(len(track_pt_arr[ievt])):
                    pt = float(track_pt_arr[ievt][itrk])
                    eta = float(track_eta_arr[ievt][itrk])
                    phi = float(track_phi_arr[ievt][itrk])
                    charge = int(track_charge_arr[ievt][itrk])
                    if charge == 0:
                        continue
                    if pt < 0.5 or abs(eta) > 2.4:
                        continue
                    if len(track_dxy_arr[ievt]) > itrk:
                        if abs(float(track_dxy_arr[ievt][itrk])) > 0.1:
                            continue
                    if len(track_dz_arr[ievt]) > itrk:
                        if abs(float(track_dz_arr[ievt][itrk])) > 0.5:
                            continue
                    hist_k_pt.Fill(pt)
                    if charge > 0:
                        kaons_plus.append((pt, eta, phi))
                    else:
                        kaons_minus.append((pt, eta, phi))
                
                for k_plus in kaons_plus:
                    for k_minus in kaons_minus:
                        pt1, eta1, phi1 = k_plus
                        pt2, eta2, phi2 = k_minus
                        dR = deltaR(eta1, phi1, eta2, phi2)
                        hist_deltaR_phi.Fill(dR)
                        if dR > 0.5:
                            continue
                        phi_mass = compute_invariant_mass(pt1, eta1, phi1, MASS_K, pt2, eta2, phi2, MASS_K)
                        if 0.9 < phi_mass < 1.2:
                            hist_phi_mass.Fill(phi_mass)
                            phi_candidates.append((phi_mass, pt1, eta1, phi1, pt2, eta2, phi2))
                
                if not phi_candidates:
                    continue
                events_with_phi += 1

                # --- RECONSTRUÇÃO DO J/ψ → μ⁺μ⁻ ---
                muons_plus = []
                muons_minus = []
                jpsi_candidates = []
                
                for imu in range(len(muons_pt[ievt])):
                    pt = float(muons_pt[ievt][imu])
                    eta = float(muons_eta[ievt][imu])
                    phi = float(muons_phi[ievt][imu])
                    charge = int(muons_charge[ievt][imu])
                    if pt < 2.0 or abs(eta) > 2.4:
                        continue
                    hist_mu_pt.Fill(pt)
                    if charge > 0:
                        muons_plus.append((pt, eta, phi, MASS_MU))
                    else:
                        muons_minus.append((pt, eta, phi, MASS_MU))
                
                for mu_plus in muons_plus:
                    for mu_minus in muons_minus:
                        pt1, eta1, phi1, m1 = mu_plus
                        pt2, eta2, phi2, m2 = mu_minus
                        dR = deltaR(eta1, phi1, eta2, phi2)
                        hist_deltaR_jpsi.Fill(dR)
                        if dR > 1.0:
                            continue
                        jpsi_mass = compute_invariant_mass(pt1, eta1, phi1, MASS_MU, pt2, eta2, phi2, MASS_MU)
                        if 2.8 < jpsi_mass < 3.4:
                            hist_jpsi_mass.Fill(jpsi_mass)
                            jpsi_candidates.append((jpsi_mass, pt1, eta1, phi1, pt2, eta2, phi2))
                
                if not jpsi_candidates:
                    continue
                events_with_jpsi += 1

                # --- RECONSTRUÇÃO DO Bₛ → J/ψφ ---
                for phi_candidate in phi_candidates:
                    phi_mass, kpt1, keta1, kphi1, kpt2, keta2, kphi2 = phi_candidate
                    for jpsi_candidate in jpsi_candidates:
                        jpsi_mass, mupt1, mueta1, muphi1, mupt2, mueta2, muphi2 = jpsi_candidate
                        bs_mass = compute_4body_invariant_mass(
                            mupt1, mueta1, muphi1, MASS_MU,
                            mupt2, mueta2, muphi2, MASS_MU,
                            kpt1, keta1, kphi1, MASS_K,
                            kpt2, keta2, kphi2, MASS_K
                        )
                        if 4.5 < bs_mass < 6.0:
                            hist_bs_mass.Fill(bs_mass)
                            events_with_bs += 1

    except Exception as e:
        print(f"Erro no arquivo {file_path}: {str(e)}")
        continue

# ---------------- Finalização ----------------
total_time = time.time() - start_time
print("\n" + "="*60)
print(f"PROCESSAMENTO CONCLUÍDO em {total_time/60:.2f} minutos")
print(f"Eventos processados: {total_events}")
print(f"Eventos com candidatos a φ: {events_with_phi}")
print(f"Eventos com candidatos a J/ψ: {events_with_jpsi}")
print(f"Eventos com candidatos a Bₛ: {events_with_bs}")

output_file.cd()
hist_phi_mass.Write()
hist_jpsi_mass.Write()
hist_bs_mass.Write()
hist_k_pt.Write()
hist_mu_pt.Write()
hist_deltaR_phi.Write()
hist_deltaR_jpsi.Write()
hist_mu_mass.Write()
hist_k_mass.Write()
output_file.Close()

print(f"Arquivo de saída criado: {output_root}")
print("Contém os histogramas:")
print("  - hist_phi_mass: Massa do φ → K⁺K⁻")
print("  - hist_jpsi_mass: Massa do J/ψ → μ⁺μ⁻")
print("  - hist_bs_mass: Massa do Bₛ → J/ψφ")
print("  - hist_k_pt: pT dos kaons")
print("  - hist_mu_pt: pT dos múons")
print("  - hist_deltaR_phi: ΔR entre kaons")
print("  - hist_deltaR_jpsi: ΔR entre múons")
print("  - hist_mu_mass: Massa dos candidatos a múon")
print("  - hist_k_mass: Massa dos candidatos a kaon")


Iniciando processamento em 2025-09-19 04:39:49

Arquivo 1/2: C0014C26-5B84-1F41-BEC4-4283C33952A9.root
    Eventos com φ: 0, com J/ψ: 0, com Bₛ: 0

Arquivo 2/2: 2B7EEF60-337B-2341-AF48-B681E2BFF486.root
    Eventos com φ: 1, com J/ψ: 0, com Bₛ: 0

PROCESSAMENTO CONCLUÍDO em 28.53 minutos
Eventos processados: 1741175
Eventos com candidatos a φ: 37
Eventos com candidatos a J/ψ: 23
Eventos com candidatos a Bₛ: 15
Arquivo de saída criado: bs_reconstruction.root
Contém os histogramas:
  - hist_phi_mass: Massa do φ → K⁺K⁻
  - hist_jpsi_mass: Massa do J/ψ → μ⁺μ⁻
  - hist_bs_mass: Massa do Bₛ → J/ψφ
  - hist_k_pt: pT dos kaons
  - hist_mu_pt: pT dos múons
  - hist_deltaR_phi: ΔR entre kaons
  - hist_deltaR_jpsi: ΔR entre múons
  - hist_mu_mass: Massa dos candidatos a múon
  - hist_k_mass: Massa dos candidatos a kaon


In [5]:
import ROOT

# Abrir arquivo ROOT
input_file = ROOT.TFile.Open("bs_reconstruction.root")  # <- esse arquivo deve ter os h_muons e h_kaons salvos antes


# Criar canvas
c = ROOT.TCanvas("c", "IsoTrack reconstructed particles", 800, 600)
#hist_kaons = input_file.Get("hist_track_charge")
#hist_kaons = input_file.Get("h_selected_kaons")
hist_kaons = input_file.Get("hist_phi_mass")

# Estilo dos histogramas
hist_kaons.SetTitle("Reconstrução da massa do J/ψ → μ⁺μ⁻ (MC) ;Massa [GeV];Eventos")
hist_kaons.Draw("HIST")
#Adicionar linha vertical na massa do Bₛ (5.366 GeV)
mass_bs = 3.0969
line = ROOT.TLine(mass_bs, 0, mass_bs, 34.5)
line.SetLineColor(ROOT.kRed)
line.SetLineWidth(2)
line.SetLineStyle(2)  # Linha tracejada
line.Draw()

# Adicionar legenda
#legend = ROOT.TLegend(0.6, 0.7, 0.85, 0.85)
#legend.AddEntry(hist_kaons, "Dados reconstruídos", "l")
#legend.AddEntry(line, "Massa do J/ψ (3.0969 GeV)", "l")
#legend.SetBorderSize(0)
#legend.Draw()



# Mostrar na tela
c.Draw()
c.Update()



Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
